# 초기화 Weight Initialization

- https://discuss.pytorch.org/t/weight-initilzation/157/9
- 작은 숫자로 초기화 하는 경우
- Xavier 초기화
- Kaiming He 초기화

In [1]:
# !pip install torch torchvision

## 1. Settings
### 1) Import required libraries

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [3]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", 
                         train = True,
                         download = True, 
                         transform = transforms.ToTensor(), 
                         target_transform = None)
mnist_test = dset.MNIST("./", 
                        train = False,
                        download = True,
                        transform = transforms.ToTensor(),
                        target_transform = None)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size = batch_size, 
                                           shuffle = True,
                                           num_workers = 2,
                                           drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size = batch_size, 
                                          shuffle = False,
                                          num_workers = 2,
                                          drop_last = True)

## 3. Model & Optimizer

### 1) CNN Model
- sigmoid 및 tanh 활성화 함수를 사용할 경우에는 Xavier 초기화를 사용하는 것이 학습에 유리합니다.
- 반면, ReLU 활성화 함수를 사용할 경우에는 He 초기화를 사용하는 것이 학습에 유리합니다.
  - 이번 예제에서는 ReLU 활성화 함수를 사용했으므로 He 초기화를 사용했습니다.

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1),    # 28 x 28
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding = 1),   # 28 x 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                  # 14 x 14
            nn.Conv2d(32, 64, 3, padding = 1),   # 14 x 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                   # 7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )             
        
        # <초기화 하는 방법>
        # 모델의 모듈을 차례대로 불러옵니다.
        for m in self.modules():
            # 만약 그 모듈이 nn.Conv2d 인 경우
            if isinstance(m, nn.Conv2d):
                '''
                # 작은 숫자로 초기화하는 방법
                m.weight.data.normal_(0.0, 0.02)     # 가중치를 평균 0, 편차 0.02로 초기화합니다.
                m.bias.data.fill_(0)                 # 편차를 0으로 초기화합니다.
                
                # Xavier 초기화
                init.xavier_normal_(m.weight.data)    # 모듈의 가중치를 xavier normal로 초기화합니다.
                m.bias.data.fill_(0)                 # 편차를 0으로 초기화합니다.
                '''
                
                # He 초기화
                init.kaiming_normal_(m.weight.data)  # 모듈의 가중치를 kaming he normal로 초기화합니다.
                m.bias.data.fill_(0)                 # 편차를 0으로 초기화합니다.
            
            # 만약 그 모듈이 nn.Linear인 경우
            elif isinstance(m, nn.Linear):
                '''
                # 작은 숫자로 초기화하는 방법
                m.weight.data.normal_(0.0, 0.02)     # 가중치를 평균 0, 편차 0.02로 초기화합니다.
                m.bias.data.fill_(0)                 # 편차를 0으로 초기화합니다.
                
                # Xavier 초기화                
                init.xavier_normal_(m.weight.data)    # 모듈의 가중치를 xavier normal로 초기화합니다.
                m.bias.data.fill_(0)                 # 편차를 0으로 초기화합니다.
                '''
                
                # He 초기화
                init.kaiming_normal_(m.weight.data)  # 모듈의 가중치를 kaming he normal로 초기화합니다.
                m.bias.data.fill_(0)                 # 편차를 0으로 초기화합니다.

    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

cuda:0


## 4. Train 

In [9]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)

tensor(2.1400, device='cuda:0', grad_fn=<NllLossBackward>)


In [10]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[-3.7371e-02,  1.7053e-01, -8.1403e-01],
          [-1.9999e-01,  4.8816e-01, -1.7883e-01],
          [ 4.7848e-01, -3.7459e-01,  5.7650e-01]]],


        [[[ 6.6440e-02, -3.8400e-01, -4.7322e-02],
          [-1.3658e+00,  4.4138e-01,  6.4916e-01],
          [ 4.1214e-01, -1.9651e-01,  1.9312e-02]]],


        [[[-2.4676e-01, -7.9292e-01, -1.3047e-01],
          [-4.3933e-01,  5.0922e-01, -5.6515e-01],
          [-5.5335e-01,  2.7607e-01,  8.7953e-01]]],


        [[[-4.2720e-01,  5.0283e-01, -1.2126e-02],
          [ 1.5341e-01, -6.7300e-01, -8.5427e-01],
          [ 1.6369e-01,  7.4674e-01,  5.9016e-01]]],


        [[[ 1.0842e+00,  7.8164e-01, -7.5962e-01],
          [-6.9601e-02, -5.0128e-01,  1.9391e-02],
          [-5.6214e-01,  2.0941e-01, -2.2193e-01]]],


        [[[ 2.0920e-01, -9.0818e-01, -9.2746e-02],
          [-4.3239e-01,  1.4653e-01, -6.8300e-01],
          [ 2.0935e-01,  1.2982e-01, -1.0257e-01]]],


        [[[-6.3084e-01, -2.1290e-01

## 5. Test

In [11]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:
      x = image.to(device)
      y_ = label.to(device)

      output = model.forward(x)
      _, output_index = torch.max(output, 1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 84.0244369506836
